### I package all the code demonstrated in the work flow file into one function. 
### You can simply download the ipynb file and execute the code.

In [5]:
import pandas as pd
import scipy.optimize as optimize

def forward_rate():
    # Step 1: ask user to input parameters
    print("What is the Current Bond Price?")
    price = float(input())
    print("What is the Bond Par Value?")
    par = float(input())
    print("What is the Bond Coupon Rate (% p.a.)")
    cr = float(input())
    print("How long is the Years to Maturity")
    n = float(input())
    print("How frequent is the Payment per year")
    freq = float(input())
    
    # Step 2: Calculate YTM of the bond base on the input of the user.
    def bond_ytm(price, par, n, cr, freq, guess=0.05):
        freq = float(freq)
        periods = n*freq
        coupon = cr/100.*par/freq
        dt = [(i+1)/freq for i in range(int(periods))]
        ytm_func = lambda y: sum([coupon/(1+y/freq)**(freq*t) for t in dt]) + par/(1+y/freq)**(freq*n) - price     
        return optimize.newton(ytm_func, guess)
    
    if __name__ == "__main__":
        ytm = bond_ytm(price=price, par=par, n=n, cr=cr, freq=freq)
        print("\n The YTM of the bond is: {}.".format(ytm))
    
    
    # Step 3: Calculate the spot rate of the bond.
    def spot(price, par, n, cr, freq, guess=0.05):
        freq = float(freq)
        periods = n*freq
        coupon = cr/100.*par/freq
        dt = [(i+1)/freq for i in range(int(periods)-1)]
        spot_func = lambda x: price - sum([coupon/(1+ytm/n/freq*t)**(freq*t) for t in dt]) - (par+coupon)/(1+x)**(freq*n)
        return optimize.newton(spot_func, guess)
    
    if __name__ == "__main__":
        s = spot(price=price, par=par, n=n, cr=cr, freq=freq)*freq
        
        
    # Step 4: Create an empty dataframe to store values.
    # produce row index for the dataframe
    row_name = list(range(int(n*freq)+1))
    row_name = [x for x in row_name]
    row_name = ['第'+ str(x) +'期' for x in row_name]
    
    df2 = pd.DataFrame(index=row_name, columns=row_name)
    df2 = df2.fillna(0) # with 0s rather than NaNs
    
    for i in list(range(len(row_name))):
        for j in list(range(len(row_name))):
            if i>j:
                df2.iloc[i,j] = str('X')
            elif i==j:
                df2.iloc[i,j] = str('0')
     
    
    # Step 5: Store the spot rate calculated above into the first row.   
    T = [(i+1)/freq for i in range(int(n*freq)-1)]
    for i in list(range(int(n*freq))):
        if i==0:
            df2.iloc[0, int(i)] = 0
        else:
            df2.iloc[0, int(i)] = [((1+ytm/n/freq/freq)**(freq*t)-1)*freq for t in T][int(i-1)]
            
    df2.iloc[0,int(n*freq)] = s
    spot_rate = df2.iloc[0,:][(-len(df2.iloc[0,:])+1):len(df2.iloc[0,:])]
    print("\n The spot rate of the bond is: \n{}.".format(spot_rate))

    
    # Step 6: Calculate the forward rate from the spot rate, and store into the dataframe.
    for i in list(range(len(row_name))):
        for j in list(range(len(row_name))):
            if j>i and i!=0:
                df2.iloc[i,j] = freq*(((1+df2.iloc[i-1,j]/freq)**j/(1+df2.iloc[i-1,j-1]/freq)**i)**(1/(j-i))-1)
    
    
    # Step 7: Demonstrate the result with two tables 
    print("\n The forward rate table show as below:")
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    display(df2)

In [6]:
forward_rate()

What is the Current Bond Price?
1029
What is the Bond Par Value?
1000
What is the Bond Coupon Rate (% p.a.)
5
How long is the Years to Maturity
2
How frequent is the Payment per year
2

 The YTM of the bond is: 0.034862654986864326.

 The spot rate of the bond is: 
第1期    0.00871566
第2期     0.0174693
第3期     0.0262611
第4期     0.0354197
Name: 第0期, dtype: object.

 The forward rate table show as below:


,第0期,第1期,第2期,第3期,第4期
第0期,0,0.00871566,0.0174693,0.0262611,0.0354197
第1期,X,0,0.0262611,0.0306714,0.0384817
第2期,X,X,0,0.0395207,0.0463221
第3期,X,X,X,0,0.0668628
第4期,X,X,X,X,0
